In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import glob
from pathlib import Path
import os
from os.path import join

In [3]:
subject = 'NP30'
block = 'B12'
subject_block = '_'.join((subject, block))
datatype='neuropixel'

In [4]:
preproc_root = Path('/userdata/smetzger/repos/np_preproc/output')
if not subject == 'NP11':
    kilosort_root = Path(f'/userdata/smetzger/data/{subject}_{block}_mc/')
else: 
    kilosort_root = Path('/userdata/smetzger/data/NP11_B4_mc_update/NP11_B4_data/NP11_B4_mc/')
labels_fp = os.path.join(preproc_root, subject_block, 'labels', 'combined_speech_labels.csv')
labels = pd.read_csv(labels_fp)

In [5]:
# Make a mapping from the feature names, to the dir within that feature names output that has results!
resdir_dict = {
    'acoustic_feats':None,
    'artics':'artic_files/F01_indep_Haskins_loss_90_filter_fix_bn_False_0_setting2',
    'formants':None,
    'phones':'results/speaker1',
    'pitch':None,
    'spectrograms':None,
}

resfeat_dict = {
    'acoustic_feats':['peakRate', 'peakEnv', 'env'],
    'artics':['ttx', 'tty', 'tdx', 'tdy', 'tbx', 'tby', 'lix', 'liy', 'ulx', 'uly', 'llx', 'lly', 'la', 
             'pro', 'ttcl', 'tbcl', 'vx', 'vy'], #Note the last one is lip aperture. 
    'formants':['f1', 'f2', 'f3'],
    'pitch':['rel-pitch', 'pitchMin', 'pitchMax', 'pitchUp', 'pitchDown'],
    'spectrograms':['spec_%d' %k for k in range(80)], # Not sure the frequency bins as of now.
    'phones':None # need to use textgrids still for these :D 
}

expected_offsets = {
    'formants':0.025
}

def time_from_resfile(file):
    """
    given a file, e.g. stim_t0_t1.mat etc
    returns the t0, t1 tuple as a set of floats.
    """
    t0 = None
    t1 = None
    if len(file.split('_')) == 3:
        t0 = float(file.split('_')[1])
        t1 = float('.'.join(file.split('_')[2].split('.')[:2]))
    
    elif len(file.split('_')) == 2:
        t0 = float(file.split('_')[0])
        t1 = float('.'.join(file.split('_')[1].split('.')[:2]))
        
    return t0, t1
    
basefp  = str(preproc_root/ subject_block)
### Adjust the timings based on whats in the files

In [6]:
# Step 1: Lets load in the neural data. 

if datatype == 'neuropixel':
    ks_out_fp = str(kilosort_root)
    spikes = pd.read_csv(join(kilosort_root, 'cluster_group.tsv'), sep='\t')
    from process_units.extract_spikes_from_kilosort import extract_spikes
    neural_array, unit_tags = extract_spikes(ks_out_fp, 
                                 30e3, 
                                  ks_offset = 0,
                                  spike_cutoff=1000,
                                    prcsd_sr = 100, 
                                        many_clusters=None
                                 )
else: 
    ks_out_fp

In [7]:
#### This is a dataframe with the phonemes and words from MFA
from preproc_func.phoneme_extract.process_textgrids import extract_phonemes, phone_df_to_arr, phone_df_to_word_level_feats
phonemes_and_words_df = extract_phonemes(os.path.join(preproc_root, subject_block, 'phones', 'results'), 
                                         100, neural_array.shape[0])
phone_features, phone_enc_dict = phone_df_to_arr(phonemes_and_words_df, 100, neural_array.shape[0], 0)
word_onsets, onset_labels, word_offsets, offset_labels = phone_df_to_word_level_feats(phonemes_and_words_df, 100, neural_array.shape[0], 0)

shape (170172, 66)


In [8]:
import matplotlib.pyplot as plt

In [9]:
from file_compilation.file_compilation import postprocess_phonemes

In [10]:
#### We can derive additional features for free based on the phonemes. 
# This will output the following features: 
# 

In [11]:
word_feats_df, sent_feats_df, phones_df, stress_df, complexity_df, vowels  = postprocess_phonemes(phonemes_and_words_df, 
                                  phone_features, phone_enc_dict, 
                                  word_onsets, 
                                  onset_labels, 
                                  word_offsets, 
                                  offset_labels, neural_array, labels, 
                                 labels_fp)

[nltk_data] Downloading package cmudict to /home/smetzger/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


saved at /userdata/smetzger/repos/np_preproc/output/NP30_B12/labels/combined_speech_labels_fa.csv


In [12]:
#Okay, now we can go through each file, and load the file into the dataframe.

In [13]:
resdir_dfs = []
for k, v in resdir_dict.items():
    print('loading', k)
    if k == 'phones':
        continue
    if v is None: 
        resfp = os.path.join(basefp, k)
    else: 
        resfp = os.path.join(basefp, k, v)
    files = sorted(os.listdir(resfp))
    files = [f for f in files if not f == 'results.mat']
    print(len(files), k)
    print(len(resfeat_dict[k]))
    featarray = np.empty((neural_array.shape[0], len(resfeat_dict[k])))
    featarray[:] = np.nan
    for f in files: 
        if not f.endswith('.npy'):
            continue
        data = np.load(os.path.join(resfp, f))
        t0, _ = time_from_resfile(f)
        ind = int(t0*100)
        end = ind + data.shape[0]
        featarray[ind:end] = data
    resdir_dfs.append(pd.DataFrame(data=featarray, columns=resfeat_dict[k]))

loading acoustic_feats
498 acoustic_feats
3
loading artics
498 artics
18
loading formants
498 formants
3
loading phones
loading pitch
498 pitch
5
loading spectrograms
498 spectrograms
80


In [14]:
features_df = pd.concat(resdir_dfs + list([word_feats_df, sent_feats_df, phones_df, stress_df, complexity_df]) 
                        , axis=1)

In [15]:
neural_df = pd.DataFrame(data=neural_array, columns = unit_tags)

In [16]:
features_df = pd.concat((features_df, neural_df), axis=1)

In [17]:
features_df['time (s)'] = np.arange(len(features_df))/100

In [ ]:
from file_compilation.file_compilation import extract_mxu_vid_feats
if subject in ['NP30', 'NP11']:
    video_derived_df = extract_mxu_vid_feats(subject)
    features_df.index = features_df['time (s)']

dict_keys(['__header__', '__version__', '__globals__', 'c_arr', 'h_arr', 'll_arr', 't_arr', 'ul_arr', 'w_arr'])


In [ ]:
if subject in ['NP30', 'NP11']:
    features_df= features_df.join(video_derived_df.set_index('t'))

In [ ]:
features_df= features_df.rename(columns= {u:'unit_%d' %u for u in unit_tags})

In [ ]:
features_df.index = np.arange(len(features_df))/100

In [ ]:
print(list((features_df.columns)))

In [ ]:
features_df.to_hdf(os.path.join(preproc_root, subject_block, subject_block+ '_feats.h5'), key='df')

In [ ]:
len(features_df)